# Design a bigger dataset of queries by using data augmentation

We want to change two things:
*  we change some filter conditions 
*  we (exhaustively) change the attribute in the MIN-aggregation, such that for each query one variable of each relation is represented once in the new dataset

Dataset size in the beginning: 203
*  STATS: 146
*  SNAP: 40
*  JOB: 15
*  LSQB: 2

#### Augmentation: Filter
By hand we change each STATS and JOB query twice with changing different filters. (SNAP and LSQB do not have any filters.)   
(The new queries are called "query"-augF1 and "query"-augF2, where "query" is the original name of the query.)   
For 6 STATS queries there is only one filter. Then we only create one new query "query"-augF1. This is the case for:
*  STATS: 024-017
*  STATS: 025-001
*  STATS: 096-095
*  STATS: 100-005
*  STATS: 111-056
*  STATS: 143-126

Therefore we get a new dataset with size: 519
*  STATS: 140 * 3 + 6 * 2 = 432
*  SNAP: 40
*  JOB: 15 * 3 = 45
*  LSQB: 2

#### Augmentation: Change aggregate-attribute
Now we take this new dataset and change the attribute in the MIN such that for each relation per query there is a query with an attribute of this table in the aggregation. The new dataset size is now the sum of the number of relations per query.   
(This will be about 2200 queries.)

In [58]:
432+40+45+2

519

In [ ]:
#%%bash
#pip install pandas

In [ ]:
import pandas as pd

In [12]:
40 + 734 + 734 +734

2242

In [52]:
df = pd.read_csv("results/features_times.csv")

In [53]:
df[df["bench"] != "SNAP"]["#relations"].sum()

734

In [54]:
203-163

40

In [55]:
df[df["#filters"] == 0]

,bench,query,orig/rewr(mean),orig/rewr+rewr(mean),orig mean,rewr mean,rewr mean+rewr,diff rewr-orig,diff rewr+rewr-orig,#relations,...,min(branching factors),max(branching factors),mean(branching factors),median(branching factors),balancedness factor,list table rows,list join rows,container counts list,branching factors list,text
146,SNAP,dblp-path02,rewr,rewr,2.294022,0.870167,2.119866,-1.423855,-0.174155,3,...,2,2,2.000000,2.0,1.000000,"[437444, 437444, 437444]","[17539780, 2769959]","[1, 1, 1, 1, 2]",[2],"select MIN(p1.toNode) from dblp p1, dblp p2, d..."
147,SNAP,dblp-path03,rewr,rewr,30.795075,1.407008,2.529921,-29.388067,-28.265154,4,...,1,2,1.500000,1.5,1.000000,"[437444, 437444, 1049866, 1049866]","[111064404, 2769959, 6647902]","[1, 1, 1, 1, 1, 1, 2]","[1, 2]","select MIN(p1.toNode) from dblp p1, dblp p2, d..."
148,SNAP,dblp-path04,rewr,rewr,387.542229,1.624270,2.630539,-385.917959,-384.911690,5,...,1,2,1.500000,1.5,1.000000,"[437444, 437444, 437444, 1049866, 1049866]","[703275764, 17539780, 2769959, 6647902]","[1, 1, 1, 1, 1, 1, 1, 1, 2]","[1, 2]","select MIN(p1.toNode) from dblp p1, dblp p2, d..."
149,SNAP,dblp-path05,rewr,rewr,1800.000000,1.857328,2.905927,-1798.142672,-1797.094073,6,...,1,2,1.500000,1.5,1.000000,"[437444, 437444, 437444, 1049866, 1049866, 104...","[4453243180, 17539780, 2769959, 42095471, 6647...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[1, 2]","select MIN(p1.toNode) from dblp p1, dblp p2, d..."
150,SNAP,dblp-path06,rewr,rewr,1800.000000,2.054539,3.087870,-1797.945461,-1796.912130,7,...,1,2,1.500000,1.5,1.000000,"[437444, 437444, 1049866, 1049866, 1049866, 10...","[28198575625, 111064404, 2769959, 6647902, 420...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[1, 2]","select MIN(p1.toNode) from dblp p1, dblp p2, d..."
151,SNAP,dblp-path07,rewr,rewr,1800.000000,2.245325,3.223917,-1797.754675,-1796.776083,8,...,1,2,1.500000,1.5,1.000000,"[437444, 437444, 1049866, 1049866, 1049866, 10...","[178557432222, 111064404, 2769959, 6647902, 26...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[1, 2]","select MIN(p1.toNode) from dblp p1, dblp p2, d..."
152,SNAP,dblp-path08,rewr,rewr,1800.000000,2.513097,3.493093,-1797.486903,-1796.506907,9,...,1,2,1.500000,1.5,1.000000,"[437444, 437444, 437444, 1049866, 1049866, 104...","[1130651314648, 703275764, 17539780, 2769959, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 2]","select MIN(p1.toNode) from dblp p1, dblp p2, d..."
153,SNAP,dblp-tree01,rewr,rewr,1435.961778,1.863914,2.910837,-1434.097864,-1433.050941,5,...,2,2,2.000000,2.0,0.666667,"[437444, 1049866, 1049866, 1049866, 1049866]","[703275764, 17539780, 2769959, 6647902]","[1, 1, 1, 1, 1, 2, 3]","[2, 2]","SELECT MIN(p1.toNode) FROM dblp p1, dblp p2, d..."
154,SNAP,dblp-tree02,rewr,rewr,1800.000000,3.441422,4.450278,-1796.558578,-1795.549722,6,...,2,3,2.500000,2.5,0.625000,"[437444, 1049866, 1049866, 1049866, 1049866, 1...","[4453243180, 17539780, 2769959, 42095471, 6647...","[1, 1, 1, 1, 1, 1, 1, 2, 3]","[3, 2]","select MIN(p1.toNode) from dblp p1, dblp p2a, ..."
155,SNAP,dblp-tree03,rewr,rewr,1800.000000,4.976418,5.984052,-1795.023582,-1794.015948,8,...,1,3,2.000000,2.0,0.625000,"[437444, 437444, 1049866, 1049866, 1049866, 10...","[178557432222, 4453243180, 111064404, 17539780...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3]","[1, 3, 2]","select MIN(p1.toNode) from dblp p1, dblp p2a, ..."


In [ ]:
df[df["bench"] 

In [59]:
730*3

2190

In [25]:
%%bash
pip install psycopg2-binary
pip install numpy

In [26]:
import json
import time
import psycopg2
import numpy as np
import csv
import multiprocessing
import signal

In [49]:
df = pd.read_csv('scala_commands_with_MIN_augment_filter.txt', delimiter=' ', quotechar='"', escapechar='\\', header = None)
df.replace('\\\\"', '', regex=True, inplace=True)

df.iloc[10][1]

"SELECT MIN(c.Id) FROM comments as c, postHistory as ph, badges as b, votes as v, users as u WHERE u.Id =b.UserId AND b.UserId = ph.UserId AND ph.UserId = v.UserId AND v.UserId = c.UserId AND c.CreationDate>=CAST('2010-07-20 21:37:31' AS TIMESTAMP) AND ph.PostHistoryTypeId=12 AND u.UpVotes=1"

In [39]:
def handler_orig(signum, frame):
    global timeout_flag_orig
    timeout_flag_orig = True
    raise Exception("Query execution of the original query > 1800s")

def handler_rewr(signum, frame):
    global timeout_flag_rewr
    timeout_flag_rewr = True
    raise Exception("Query execution of the rewritten query > 1800s")
    
def run_query(benchmark, query):

    original_query = query 
    database = benchmark.lower()
    conn = psycopg2.connect(
        host="postgres",
        database=database,
        user=database,
        password=database
    )

    # if the evaluation takes longer than 30min then break it
    global timeout_flag_orig
    timeout_flag_orig = False

    print("original1")
    # the first run is just a warm up run and to check for the time out
    signal.signal(signal.SIGALRM, handler_orig) 
    signal.alarm(1800) 
    try:
        print("o1")
        cur = conn.cursor()
        print("o2")
        cur.execute(original_query)
        print("o3")
        result = cur.fetchall()
    except Exception as exc: 
        print(exc)
    signal.alarm(0) 

    if not timeout_flag_orig:
        list_original = []
        print("orig")
        for i in range(5):
            # execute the original query
            start_time_original = time.time()
            cur.execute(original_query)
            end_time_original = time.time()
            original_time = end_time_original - start_time_original
            list_original.append(original_time)
        orig_mean = np.mean(list_original)
        orig_med = np.median(list_original)
        orig_std = np.std(list_original)
        
            
    list_output = [benchmark, query] + [orig_mean] + list_original + [orig_med, orig_std] 
    #print(list_output)
    file_path = "results/POS_Scala_comparison_TO_augment_filter.csv"
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(list_output)

In [27]:
file_path = "results/POS_Scala_comparison_TO_augment_filter.csv"

names = ["bench", "query", "orig mean", "orig 1", "orig 2", "orig 3", "orig 4", "orig 5", "orig med", "orig_std"]

with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(names)

In [50]:
for index, row in df.iterrows():
    run_query("STATS", row[1])

original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig
original1
o1
o2
o3
orig


KeyboardInterrupt: 

In [45]:
run_query("STATS", 'SELECT MIN(pl.Id) FROM postLinks as pl, posts as p, users as u, badges as b WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND pl.CreationDate<=CAST(\'2011-08-17 01:23:50\' AS TIMESTAMP) AND p.Score>=-1 AND p.Score<=10 AND p.AnswerCount<=5 AND p.CommentCount=2 AND p.FavoriteCount>=0 AND p.FavoriteCount<=6 AND u.Views<=33 AND u.DownVotes>=0 AND u.CreationDate>=CAST(\'2010-08-19 17:31:36\' AS TIMESTAMP) AND u.CreationDate<=CAST(\'2014-08-06 07:23:12\' AS TIMESTAMP) AND b.Date<=CAST(\'2014-09-10 22:50:06\' AS TIMESTAMP)')

original1
o1
o2
o3
orig


In [35]:
# Open the CSV file in read mode
with open('results/POS_Scala_comparison_TO.csv', 'r') as file:
    lines = file.readlines()  # Read all lines into a list

# Exclude the last 9 lines
modified_lines = lines[:-9]

# Open the CSV file in write mode and overwrite it with the modified content
with open('results/POS_Scala_comparison_TO.csv', 'w') as file:
    file.writelines(modified_lines)
